# Co to jest PySpark?

PySpark to biblioteka, której można używać do uruchamiania aplikacji Pythona przy użyciu możliwości Apache Spark. Innymi słowy, PySpark to API Pythona dla Sparka. Wiele osób myli Sparka z językiem programowania. Spark też jest biblioteką, której mogą używać języki programowania takie jak Java, Scala, R i Python. Używając PySpark możemy uruchamiać aplikacje równolegle w rozproszonym klastrze. Klaster to zestaw komputerów w jednej sieci. No to po co ten Spark? 

Zacznijmy najpiew od Apache Spark. Jest to silnik stworzony do wydajnego rozproszonego, wielkoskalowego przetwarzania danych, a także obsługiwania aplikacji uczenia maszynowego. Spark jest odpowiedzią na potrzebę biznesową związaną z przetwarzaniem ogromnych wolumentów danych. Jedną z pierwszych odpowiedzi na tę potrzebę było wynalezienie Hadoopa, a także Mapreduce. Hadoop to zestaw bibliotek zaprojektowanych do działania w klastrze, a Mapreduce to właściwy silnik przetwarzania danych. Problem z Mapreduce polega na tym, że wykonuje on większość operacji na dysku, co skutkowało dłuższym czasem oczekiwania na wynik. Dlatego właśnie w 2009 r. na Uniwersytecie Berkeley wynaleziono Sparka. Spark pracuje na dysku 10 razy szybciej niż Mapreduce i 100 razy szybciej w pamięci. I dlatego właśnie Spark zyskał na popularności

# Spark Unified Stack 

Teraz porozmawiajmy o tzw. Spark Unified Stack. Spark Unified Stack jest zbudowany na Spark Core. Spark Core jest nośnikiem wszystkich funkcje niezbędnych do zarządzania i uruchamiania aplikacji rozproszonych. Mówimy o: planowaniu, koordynacji i odporności na awarie. W pracy ze Sparkiem wykorzystywanych jest kilka modułów i komponentów i każdy z nich jest oparty o Spark Core. Są to: 
* Spark SQL - ten moduł jest przeznaczony do pracy z danymi ustrukturyzowanymi. Pozwala odczytywać dane z tabel relacyjnej bazy danych lub z CSVek, JSONów, Avro, ORC, a także plików parquet. Można nawet pobierać i zapisywać dane do sparkowje ramki danych. Spark SQL jest zgodny ze standardem ANSI SQL, co oznacza, że działa również SQLengine, umożliwiając pracę przy użyciu składni SQL. 
* Spark Structured Streaming - umożliwia przetwarzanie strumieniowych danych w czasie rzeczywistym z różnych źródeł z wysoką przepustowością i odpornością na błędy. Dane mogą być pobierane z Kafki, Flume'a, Twitter'a itd. i przetwarzane w czasie rzeczywistym. 
* Spark MLlib - moduł do uczenia maszynowego. W jego skład wchodzi wiele popularnych algorytmów uczenia maszynowego, które pozwalają budować pipeline'y do uczenia i oceny modeli.  
* Spark GraphX - moduł do tworzenia grafów reprezentujących sieci, takich jak LinkedIn.

# Spark web UI

Apache Spark zapewnia interfejs sieciowy do monitorowania stanu wykonywanej w klastrze aplikacji. Żeby to zrozumieć najlepiej spojrzeć na przykład. Stworzymy rdd, które przyjmie listę liczb. Każdą z tych liczb podniesiemy do kwadratu przy wykorzystaniu wyrażenia lambda. Tworzymy rdd, więc musimy użyć funkcji (a dokładniej: transofrmacji) specyficznej dla RDD: *map()*. Mapowania używa się do złożonych operacji na danych. 
Zaczniemy do inicjalizacji sesji: 

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
# Może chwilę potrwać
spark = SparkSession.builder.appName("ReadWriteVal").getOrCreate()
spark

Teraz stworzymy ramkę:

In [2]:
num_list = [1,2,3,4]
rdd = spark.sparkContext.parallelize(num_list) # o funkcji parallelize() porozmawiamy potem
squared = rdd.map(lambda x: x**2).collect() # o funkcji collect() porozmawiamy potem

#sprawdźmy czy wszystko ok
for num in squared: 
    print(num)

1
4
9
16


In [3]:
# podejrzyjmy typ obiektu: 
type(rdd)

pyspark.rdd.RDD

Teraz uruchomimy interfejs poprzez kliknięcie w tekst Spark UI, który widoczny jest tuż pod miejscem, w którym zainicjalizowaliśmy sesję. Pierwsza rzecz, która rzuca się w oczy to to, że interfejs użytkownika Spark jest podzielony na kilka zakładek: Jobs, Stages, Storage, Environement, Executors, SQL. Zwróc uwagę na nazwę aplikacji, jest zgodna z tym co jej przypisaliśmy (lub nie) na etapie inicjalizacji sesji. To, co jest ważny to to, że każdy program, który napiszesz, będzie miał przypisany własny adres URL.  

Wróćmy do sekcji Jobs. Można w nim zdefiniować czy podejrzeć tryb planowania, liczbę zadań Spark, liczbę etapów zadania oraz opisy zadań. Jeśli chodzi o planowanie dostępne są trzy tryby planowania: autonomiczny (FIFO: first in, first out). Drugi tryb **YARN**. Trzeci to tryb **Mesos**. Przy planowaniu zadań, które nie będą wykorzystywać przetwarzania rozproszonego (tak jak my teraz, na warsztatach) YARN i Mesos są niepotrzebne. Działając lokalnie będziemy wykorzystywać tylko tryb FIFO. 

Rzućmy teraz okiem na spójrz na Spark Jobs. Do tej pory wykonaliśmy tylko jednego Joba. Spark Job to liczba **akcji** w aplikacji. Będziemy mówić o **akcjach i transformacjach** trochę później. To, co teraz należy wiedzieć to to, że są to dwa całkowicie odrębne procesy i różnice między nimi, z punktu widzenia ewaluacji kodu Sparka, są bardzo istotne. W UI widać tylko **akcję** *collect()*., o której wiemy, że poprzedziliśmy ją **transformacją** *map()*. Dla Sparka jest to pojedynczy etap wykonywanego zadania (Job). Każdy Job w Sparku powinien mieć przynajmniej jeden etap. Jednak nie wszystkie operacje będą się kończyły na jednym. W rzeczywistości zadanie może mieć jeden lub więcej etapów. Etapy w Sparku powstają zależnie od tego czy operacje można wykonywać szeregowo czy równolegle. Każdy etap może mieć jedno lub więcej mniejszych zadań (task). Etapy mogą reprezentować pojedynczy element pracy, którą musimy wykonać. Przykładowo - jeden etap mógłby reprezentować połączenie z bazą danych, inny reprezentowałby tasowanie danych i tak dalej. Gdybyśmy wykonali więcej działań pojawiłyby się w tej sekcji. 

Teraz spójrzmy na oś czasu. Unaocznia ona flow zadania - informuje o momencie inicjalizacji drivera i wykonania *collect()*. Oś czasu dostępna jest dla nas tak długo jak długo trwa dana sesja Sparka. 

Etapy (stages) - ta sekcja nam bardziej szczegółowy widok etapów wykonywanego zadania. Jeżeli klikniemy Description uzyskamy bardziej szczegółowy opis danego etapu (podrzędne w stosunku do etapów są taski).  Pierwszą rzeczę, którą zobaczymy jest wizualizacja DAG. DAG oznacza Directed Acycle Graph. Opisuje on metodologię wykonania zadań przez Sparka.  W tym momencie nie ma sensu mocno zagłębiać się w DAGi, ponieważ bierząca aplikacja jest po prostu zbyt prosta. 

Teraz rzućmy okiem na **Aggregated metrics by executor**, które znajduje się pod DAGami. Aby zrozumieć executory, wróćmy do Stage'y. Stage może utworzyć jeden lub więcej task. Każdy task jest odrębną jednostką. Taka jednostka jest przekazywana/mapowana do executora (węzła), który to zadanie wykona. W przypadku naszej prostej aplikacji, widać, że mamy jeden executor, który utworzył co najmniej osiem zadań do równoległego wykonywania. Osiem zadań odpowiada liczbie rdzeni procesora. 

# Spark Execution Concepts

Zanim przejdziemy do podstawowego obiektu Sparka, jakim są RDDsy, przytoczymy podstawowe informacje o tym, jak Spark wykonuje polecenia. Poznamy aplikacje i sesje sparkowe. Skonfigurujemy nowy projekt, w którym stworzymy aplikację obliczającą największą ilość zamówień w kraju i w regionie. A potem przejdziemy do rozmów o transformacjach i akcjach Sparka. Potem pokażemy jak interpretować DAGi. 

Na początek zaczniemy od stworzenia nowej sesji i importu funkcji sqlowej:

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import count

spark = SparkSession.builder.appName("TotalOrders").getOrCreate() # Może być tylko jedna sesja Spark per JVM. 
# Sesja sparka to instancja Javy, bo Spark został napisany w Javie
# builder to klasa, a klasy dają dotęp do funkcji charakterystycznych dla klasy 
# appName = funkcja przyjmuje tylko jeden parametr
spark

Zdecydowaliśmy się na użycie Spark SQL ze względu na to, że łatwo jest się nim posługiwać, dużo łatwiej niż. Ponadto SparkSQL zawiera w sobie proste do zrozumienia funkcje sparkowe. 

Jednak najważniejszą rzeczą, którą powinniście wynieść z tej seksji jest to, że fundamentem każdej aplikacji sparkowej jest Spark Driver, który tworzy obiekt SparkSession. Driver ułatwia także komunikację między sesją, wykorzystywanym notebookiem a executorami. 

Jesteśmy coraz bliżej zrozumienia w jaki sposób Spark wykonuje polecenia. Nim to nastąpi musimy załadować dane, żeby mieć na czym pracować.

Poniżej przykład jak wczytywać dane z plików csv. Parametr *inferSchema* ustawiony na true informuje Sparka, że ma wczytać dane w tej samej postaci, w której zostały zapisane. Parametr *header* ustawiony na true oznacza, że pierwszy wiersz wczytywanego pliku jest nagłówkiem ramki, którą właśnie tworzymy. 

In [2]:
sales_df = spark.read.csv("sales_records.csv", inferSchema=True, header=True)
# opcja alternatywna:
# sales_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("sales_df")

Podjerzyjmy sobie 10 wierszych wierszy:

In [3]:
sales_df.show(10) #to wygląda brzydko

+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|    Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+-------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Middle East and N...|          Azerbaijan|       Snacks|       Online|             C| 10/8/2014|535113847|10/23/2014|       934|    152.58|    97.44|    142509.72|  91008.96|    51500.76|
|Central America a...|              Panama|    Cosmetics|      Offline|             L| 2/22/2015|874708545| 2/27/2015|      4551|     437.2|   263.33|    1989697.2|1198414.83|   791282.37|
|  Sub-Saharan Africa|Sao Tome and Prin...|       Fruit

Skrypt poniżej również pozwoli na podejrzenie danych, ale wynik będzie lepiej ułożony:

In [4]:
sales_df.select("Region", "Country", "Order ID").show(n=10, truncate= False)

+---------------------------------+---------------------+---------+
|Region                           |Country              |Order ID |
+---------------------------------+---------------------+---------+
|Middle East and North Africa     |Azerbaijan           |535113847|
|Central America and the Caribbean|Panama               |874708545|
|Sub-Saharan Africa               |Sao Tome and Principe|854349935|
|Sub-Saharan Africa               |Sao Tome and Principe|892836844|
|Central America and the Caribbean|Belize               |129280602|
|Europe                           |Denmark              |473105037|
|Europe                           |Germany              |754046475|
|Middle East and North Africa     |Turkey               |772153747|
|Europe                           |United Kingdom       |847788178|
|Asia                             |Kazakhstan           |471623599|
+---------------------------------+---------------------+---------+
only showing top 10 rows



Upewnijmy się, że to ramka sparkowa:

In [5]:
type(sales_df)

pyspark.sql.dataframe.DataFrame

Teraz wykonamy grupowanie, agregację i zliczanie - chcemy się dowiedzieć, które państwa w obrębie danych regionów miały największą liczbę zamówień: 

In [6]:
count_sales_df = (sales_df.select("Region", "Country", "Order ID")).groupBy("Region", "Country").agg(count("Order ID").alias("Total Orders")).orderBy("Total Orders", ascending = False)

count_sales_df.show(10, False)
print("Total rows: ", count_sales_df.count())

+---------------------------------+-------------+------------+
|Region                           |Country      |Total Orders|
+---------------------------------+-------------+------------+
|Sub-Saharan Africa               |Sudan        |623         |
|Australia and Oceania            |New Zealand  |593         |
|Europe                           |Vatican City |590         |
|Europe                           |Malta        |589         |
|Sub-Saharan Africa               |Mozambique   |589         |
|Middle East and North Africa     |Tunisia      |584         |
|Asia                             |Cambodia     |584         |
|Central America and the Caribbean|Panama       |578         |
|Sub-Saharan Africa               |Rwanda       |576         |
|Sub-Saharan Africa               |Cote d'Ivoire|575         |
+---------------------------------+-------------+------------+
only showing top 10 rows

Total rows:  185


# Akcje i transformacje

Teraz możemy w końcu przejść do **akcji** i **transformacji**. Spark ma dwa rodzaje transformacji, wąskie i szerokie. 
Wąska transofmracja jest wtedy, kiedy dane wyjściowe można przeliczyć korzystając tylko z jednej partycji. Partycja, czyli węzeł w klastrze. Dane mogą być rozdystrybuowane po 200 rekordów w jednej partycji, 300 w innej itd. Korzystanie z jednej partycji oznacza, że nie musimy tasować danych zawartych na innych partycjach żeby otrzymać oczekiwany rezultat. 
Rzućmy okiem na kawałek kodu:

In [7]:
big_list = range(1000)
rdd = spark.sparkContext.parallelize(big_list,2)
odds = rdd.filter(lambda x:x % 2 !=0)
odds.take(5)

[1, 3, 5, 7, 9]

Wykonaliśmy filtrowanie wyników, co jest wąską transformacją. Przefiltrowana zostanie każda partycja osobno i każda osobno zwróci wynik. Dopiero na późniejszym etapie zostaną połączone w jeden zbiór. Partycje nie musiały wymieniać się danymi żeby zrealizować polecenie. 

A co z GroupBuy i OrderBy? GroupBy i OrderBy są przekształceniami szerokimi - zmuszają driver do przegrupowania danych między węzłami. 

Powyższa ramka przechowuje dane w dwóch partycjach. Żeby wykonać *groupBy()* czy *orderBy()* dane musiałyby zostać przetasowane żeby móc wykonać to polecenie. Po wykonaniu polecenia zostałyby zebrane do pojedynczego zbioru. Tasowanie danych to działanie mające na celu połączenie danych i ułożenie ich w nową partycję. 

Transformacje zawsze skutkują nową ramką danych, akcje nie. Ta informacja będzie nam potrzebna w kolejnej części: interpretacji DAGów. 

# Wizualizacje DAG

Witajcie dobrzy ludzie i witajcie z powrotem. W porządku, nadszedł czas, aby ponownie przyjrzeć się interfejsowi Spark Web, a następnie omówić ukierunkowany wykres acykliczny, aby uzyskać jasne wyobrażenie o tym, jak faktycznie działa silnik Spark. Możemy zacząć od odświeżenia naszego notatnika i następnie załaduj interfejs sieciowy Spark. Aby to zrobić, ponownie wybiorę „Jądro” i wybierz Uruchom ponownie i uruchom wszystko, tak po prostu. I zaczekajmy, aż zakończy wykonywanie. [Brak dźwięku] W porządku, więc wydaje się, że wykonanie zostało zakończone, a co możemy zrobić, to po prostu załadować localhost i 4040 i nacisnąć enter. Idealnie. [Brak dźwięku] W porządku. Teraz, jak widać, w przeciwieństwie do wcześniej, mamy teraz więcej rzeczy do zrobienia. Nasza aplikacja ma wiele zadań, jak widać, wiele etapów i wiele zadań, które trwają do około 200 znaków. Jednak to, co nas interesuje dla tego wykładu jest to, co znajdziemy w zakładce Etapy. Wybierzmy więc z menu zakładkę Etapy, a więc wybierz Etapy. Teraz, jak widać, mamy co najmniej osiem etapów i wspomniałem z poprzednia lekcja, że każdy etap reprezentuje jednostkę pracy. Nie pozwól, aby nazewnictwo etapów w Javie cię zniechęciło, nawet jeśli napisałeś kod w Pythonie. Należy zrozumieć, że Spark jest napisany skalarnie, który następnie kompiluje się do kodu bajtowego Javy. Twórcy Sparka zapewnili to, udostępniając API Pythona dla Sparka. Ale za kulisami dzieje się tak, że Twój kod Pythona faktycznie uzyskuje dostęp do API Javy poprzez biblioteki Py4J, które zainstalowałeś podczas instalacji PySpark. oparte na nazewnictwie etapów, aby zobaczyć Py4J w akcji wybierz łącze szczegółów na dowolnym etapie, a ja zrobię to na etapie zero, tak po prostu. I tutaj widzimy Py4J w działaniu komunikujący się z interfejsem API Java. Należy więc pamiętać o jednej rzeczy, te etapy mogą działać równolegle lub szeregowo w zależności od jednostki pracy. Od razu widać, że etapy opowiadają historię naszego programu. Był czas, kiedy potrzebowaliśmy załadować dane, był czas, kiedy potrzebowaliśmy agregacji używając funkcji konta, a na koniec musieliśmy wydrukować dane wyjściowe. W ostatniej lekcji mówiliśmy o transformacjach i akcjach Spark. Spark ma model leniwego wykonywania. Zasadniczo oznacza to, że piszesz transformacje, ale one faktycznie działają dopóki nie wywołasz akcji. Przyjrzyjmy się naszemu programowi. Właściwie możesz zobaczyć liczbę przekształceń, takich jak Select, GroupBy i OrderBy. Jednak w prawdziwie leniwym modelu Spark, nie są one wykonywane do czasu wywołania akcji, takiej jak podczas wykonywania programu Show and Count. Każda akcja zostanie uruchomiona zadanie Spark, a każde zadanie uruchomi kilka etapów i tak dalej. Zacznijmy więc od etapu zerowego i zobaczmy jednostki pracy. Zamierzam więc wybrać [Brak dźwięku] opis etapu zerowego, [Brak dźwięku] i Zamierzam rozszerzyć wizualizację DAG tutaj. DAG pozwala więc zobaczyć, jak jednostka pracy została wykonana w formie wizualnego wykresu. Tutaj widzimy skanowanie pliku, które miało miejsce, gdy załadowaliśmy plik CSV zawierający dane sprzedaży. Następnie następna część pracy polega na tym, że Spark załadował dane do RDD. Pamiętaj, że ramka danych to RDD pod spodem, ale to nie mówi całej historii. Wróćmy do zadań, wybierając powiązane zadanie z tym etapem. Teraz, aby wybrać powiązane zadania, wystarczy przewinąć z powrotem tutaj i po prostu wybrać to zadanie zero, i pozwolić mi ponownie rozwinąć wizualizację DAG. Teraz widzimy tutaj, że zaraz po zeskanowaniu CSV, Spark wywołuje inną akcję, która nazywa się WholeStageCodegen.Teraz jest to generator kodu, który konwertuje kod Spark SQL na kod bajtowy Java e w celu poprawy wydajności wykonywania w klastrze. I wtedy tylko kod zapisze plik CSV w ramce danych lub być może RDD, który jest zwykle podzielony na partycje. Oczywiście ponownie odwiedzimy DAG na późniejszym etapie. teraz zapoznaj się ze szczegółami i dowiedz się więcej. To tyle ode mnie i do zobaczenia w następnym. Do widzenia. [Brak dźwięku]